## Well the weather outside is frightful

In [2]:
from pathlib import Path
import numpy as np

In [58]:
blizzard = Path('blizzard.txt').read_text().split('\n')
blizzard = np.array([[char for char in line] for line in blizzard])

In [59]:
horiz_cycle = len(blizzard[0]) - 2
vert_cycle = len(blizzard) - 2

bliz_cycle = np.lcm(horiz_cycle, vert_cycle)

In [60]:
def update_blizzard(minute, blizzard = blizzard, 
                    horiz_cycle = horiz_cycle, vert_cycle = vert_cycle):

    ys, xs = np.where(np.isin(blizzard, ['>', '<', 'v', '^']))

    new_blizzard = np.copy(blizzard)
    new_blizzard[(ys, xs)] = '.'

    for y, x in zip(ys, xs):
        blizz = blizzard[y, x]

        if blizz == '>':
            ny, nx = [y, (x + minute)%horiz_cycle]
        elif blizz == '<':
            ny, nx = [y, (x-minute)%horiz_cycle]

        elif blizz == 'v':
            ny, nx = [(y+minute)%vert_cycle, x]
        elif blizz == '^':
            ny, nx = [(y-minute)%vert_cycle, x]

        if nx == 0:
            nx = horiz_cycle

        if ny == 0:
            ny = vert_cycle

        if new_blizzard[ny, nx] == '.':
            new_blizzard[ny, nx] = blizz
        elif new_blizzard[ny, nx] in ['>', '<', 'v', '^']:
            new_blizzard[ny, nx] =  2
        elif new_blizzard[ny, nx].isnumeric():
            new_blizzard[ny, nx] = int(new_blizzard[ny, nx]) + 1
            
    return new_blizzard
        

In [61]:
blizzard_maps = {i:update_blizzard(i) for i in range(bliz_cycle)}

In [62]:
def find_legal_moves(y, x, bliz_timer, bliz_cycle = bliz_cycle, blizzard_maps = blizzard_maps):
    next_time = (bliz_timer + 1) % bliz_cycle
    
    possible_moves = [(y, x), (y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    blizz = blizzard_maps[next_time]
    height, width = blizz.shape
    
    moves = [(my, mx) for my,mx in possible_moves if 
             my >= 0 and mx >= 0 and mx < width and my < height and blizz[my, mx] == '.']
    
    return moves, next_time

def find_prev_moves(y, x, bliz_timer, bliz_cycle = bliz_cycle, blizzard_maps = blizzard_maps):
    assert(blizzard_maps[bliz_timer][y,x] == '.')
    
    prev_time = (bliz_timer - 1) % bliz_cycle
    
    possible_moves = [(y, x), (y+1, x), (y-1, x), (y, x+1), (y, x-1)]
    
    blizz = blizzard_maps[prev_time]
    height, width = blizz.shape
    
    moves = [(my, mx) for my,mx in possible_moves if 
             my >= 0 and mx >= 0 and mx < width and my < height and blizz[my, mx] == '.']
    
    return moves, prev_time

In [68]:
ex = np.where(blizzard[-1] == '.')[0][0]
ey = blizzard.shape[0]-1

scores = {(ey,ex,time): 0 for time in blizzard_maps}

current_score = 0

last_positions = [(y,x,time) for y,x,time in scores if scores[y,x,time] == 0]

while (0, 1, 0) not in scores:
    for y,x,time in last_positions:
        p_moves, p_time = find_prev_moves(y, x, time)
        for py,px in p_moves:
            if (py, px, p_time) not in scores:
                scores[py, px, p_time] = current_score + 1

    current_score += 1

    last_positions = [(y,x,time) for y,x,time in scores if scores[y,x,time] == current_score]

In [69]:
scores[0, 1, 0]

286

## Part 2: Let it snow

In [70]:
# Start at [0, 1, 0] go to [ey, ex, time1] go back to [0, 1, time1+time2] go to [ey, ex, time1 + time2 + time3]

In [72]:
def time_to_traverse(start_y, start_x, dest_y, dest_x, start_time, bliz_cycle = bliz_cycle):
    scores = {(dest_y,dest_x,time): 0 for time in blizzard_maps}
    
    start_time = start_time % bliz_cycle
    
    current_score = 0

    last_positions = [(y,x,time) for y,x,time in scores if scores[y,x,time] == 0]

    while (start_y, start_x, start_time) not in scores:
        for y,x,time in last_positions:
            p_moves, p_time = find_prev_moves(y, x, time)
            for py,px in p_moves:
                if (py, px, p_time) not in scores:
                    scores[py, px, p_time] = current_score + 1

        current_score += 1

        last_positions = [(y,x,time) for y,x,time in scores if scores[y,x,time] == current_score]
        
    return scores[start_y, start_x, start_time]

In [73]:
ex = np.where(blizzard[-1] == '.')[0][0]
ey = blizzard.shape[0]-1

time1 = time_to_traverse(0, 1, ey, ex, 0)
time2 = time_to_traverse(ey, ex, 0, 1, time1)
time3 = time_to_traverse(0, 1, ey, ex, time1+time2)

In [75]:
time1 + time2 + time3

820